# Network Intrusion Detection System (NIDS)
## Notebook 3: Model Training & Evaluation

**Team Member:** Member 3  
**Dataset:** CIC-IDS2017 (Multi-class Classification)  
**Date:** November 24, 2025  

**Objectives:**
1. Load preprocessed data from Notebook 2
2. Train 3+ classification models
3. Generate parity plots for train and test data
4. Compute classification metrics (Accuracy, Precision, Recall, F1-Score)
5. Create confusion matrices and ROC curves
6. Compare model performance

** Professor Requirements Covered:**
- Requirement #3: Define I/O variables, implement MLR/Logistic Regression/SVC/PCA
- Requirement #4: Train ML models
- Requirement #5: Show parity plots and compute metrics

---

## 1. Import Libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
import json

# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Evaluation Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import time
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(" Libraries imported successfully!")

In [ ]:
# ============================================================================
# LOCAL OUTPUT SAVER (for Colab VS Code Extension)
# ============================================================================
# This ensures all outputs are saved to your local machine
# ============================================================================

import os
from pathlib import Path

# Detect if running on Colab
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in str(get_ipython())

if IN_COLAB:
    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        
        # Set base path to your local project in Drive
        # IMPORTANT: Update this path to match your Google Drive structure
        BASE_PATH = '/content/drive/MyDrive/MLCEProject'
        
        # Create output directories if they don't exist
        for dir_name in ['outputs', 'models', 'data']:
            Path(f'{BASE_PATH}/{dir_name}').mkdir(parents=True, exist_ok=True)
        
        print("✓ Google Drive mounted")
        print(f"✓ Base path: {BASE_PATH}")
        print(f"✓ Outputs will save to: {BASE_PATH}/outputs")
        print(f"✓ Models will save to: {BASE_PATH}/models")
        print(f"✓ Data will save to: {BASE_PATH}/data")
        
    except Exception as e:
        print(f"⚠️  Could not mount Drive: {e}")
        print("Using Colab local storage (will not sync automatically)")
        BASE_PATH = '/content'
else:
    # Running locally - use relative paths
    BASE_PATH = '..'
    print("✓ Running locally")
    print("✓ Using relative paths (../outputs, ../models, ../data)")

# Helper functions for saving with correct paths
def get_output_path(filename):
    """Get correct path for output file"""
    return f"{BASE_PATH}/outputs/{filename}"

def get_model_path(filename):
    """Get correct path for model file"""
    return f"{BASE_PATH}/models/{filename}"

def get_data_path(filename):
    """Get correct path for data file"""
    return f"{BASE_PATH}/data/{filename}"

print("\n✓ Local save helper ready!")
print("\nUse these functions to save files:")
print("  - get_output_path('plot.png')  → saves to outputs/")
print("  - get_model_path('model.pkl')  → saves to models/")
print("  - get_data_path('data.csv')    → saves to data/\n")


In [ ]:
# Constants
RANDOM_SEED = 42
N_JOBS = -1
DPI = 150  # For plot saving

print("✓ Constants defined")


---
## 2. Load Preprocessed Data

Load the train and test sets generated by Member 2 in Notebook 2.

In [ ]:
# Load preprocessed data with memory optimization
print("Loading preprocessed data...\n")

# OPTIMIZATION: Use dtype specification and chunked reading for large files
print("Loading X_train...")
X_train = pd.read_csv('../data/X_train.csv', dtype='float32')  # Use float32 to save memory
print(f"  Shape: {X_train.shape}")

print("Loading X_test...")
X_test = pd.read_csv('../data/X_test.csv', dtype='float32')
print(f"  Shape: {X_test.shape}")

print("Loading y_train...")
y_train = pd.read_csv('../data/y_train.csv').values.ravel()
print(f"  Shape: {y_train.shape}")

print("Loading y_test...")
y_test = pd.read_csv('../data/y_test.csv').values.ravel()
print(f"  Shape: {y_test.shape}")

print("\n" + "="*70)
print("DATA LOADED SUCCESSFULLY")
print("="*70)
print(f"Training set:   {X_train.shape}")
print(f"Test set:       {X_test.shape}")
print(f"Classes:        {np.unique(y_train)}")
print(f"Num classes:    {len(np.unique(y_train))}")
print(f"Memory usage:   {(X_train.memory_usage(deep=True).sum() + X_test.memory_usage(deep=True).sum()) / 1024**2:.2f} MB")
print("="*70)

# Load label mapping
with open(get_output_path('label_mapping.json', 'r') as f:
    label_mapping = json.load(f)

print("\nLabel Mapping:")
for label, code in label_mapping.items():
    print(f"  {code}: {label}")


---
## 3. Define Input-Output Variables

** Professor Requirement #3: Identify input-output variables and choose classification**

In [ ]:
print("="*70)
print("INPUT-OUTPUT VARIABLE DEFINITION")
print("="*70)
print(f"\n**Problem Type:** Multi-class Classification")
print(f"\n**Input Variables (X):**")
print(f"  - Number of features: {X_train.shape[1]}")
print(f"  - Feature names (first 10):")
for i, col in enumerate(X_train.columns[:10], 1):
    print(f"      {i:2d}. {col}")
print(f"      ... ({X_train.shape[1] - 10} more features)")

print(f"\n**Output Variable (y):**")
print(f"  - Variable name: Label_Encoded")
print(f"  - Type: Categorical (Multi-class)")
print(f"  - Number of classes: {len(np.unique(y_train))}")
print(f"  - Class labels: {list(label_mapping.keys())}")

print(f"\n**Models to Implement:**")
print(f"  1. Logistic Regression (Multi-class, multinomial)")
print(f"  2. Support Vector Classifier (SVC, RBF kernel)")
print(f"  3. PCA + Logistic Regression (Dimensionality reduction)")
print("="*70)

---
## 4. Model 1: Logistic Regression (Multi-class)

** Professor Requirement #4: Train ML models**

Train Logistic Regression with multinomial setting for multi-class classification.

In [ ]:
print("="*70)
print("MODEL 1: LOGISTIC REGRESSION (Multi-class)")
print("="*70)
print("\nTraining Logistic Regression...\n")

start_time = time.time()

# Initialize Logistic Regression
lr_model = LogisticRegression(
    multi_class='multinomial',  # For multi-class classification
    solver='saga',              # Recommended for multinomial
    max_iter=100,  # SAGA solver needs more iterations
    tol=1e-3,                    # Slightly relaxed for speed
    random_state=RANDOM_SEED,
    n_jobs=N_JOBS,                    # Use all CPU cores
    verbose=1
)

# Train model
lr_model.fit(X_train, y_train)

train_time = time.time() - start_time

# Store for comparison
lr_train_time = train_time
print(f"✓ Training completed in {train_time:.2f} seconds")

# Predictions
print("\nGenerating predictions...")
y_train_pred_lr = lr_model.predict(X_train)
y_test_pred_lr = lr_model.predict(X_test)

# Predict probabilities for parity plots
y_train_proba_lr = lr_model.predict_proba(X_train)
y_test_proba_lr = lr_model.predict_proba(X_test)

print(f"\nPrediction shapes:")
print(f"  Train predictions: {y_train_pred_lr.shape}")
print(f"  Test predictions:  {y_test_pred_lr.shape}")
print(f"  Train probabilities: {y_train_proba_lr.shape}")
print(f"  Test probabilities:  {y_test_proba_lr.shape}")


In [ ]:
# Compute metrics
lr_train_metrics = {
    'Accuracy': accuracy_score(y_train, y_train_pred_lr),
    'Precision (macro)': precision_score(y_train, y_train_pred_lr, average='macro', zero_division=0),
    'Recall (macro)': recall_score(y_train, y_train_pred_lr, average='macro', zero_division=0),
    'F1-Score (macro)': f1_score(y_train, y_train_pred_lr, average='macro', zero_division=0)
}

lr_test_metrics = {
    'Accuracy': accuracy_score(y_test, y_test_pred_lr),
    'Precision (macro)': precision_score(y_test, y_test_pred_lr, average='macro', zero_division=0),
    'Recall (macro)': recall_score(y_test, y_test_pred_lr, average='macro', zero_division=0),
    'F1-Score (macro)': f1_score(y_test, y_test_pred_lr, average='macro', zero_division=0)
}

print("\n" + "="*70)
print("LOGISTIC REGRESSION - METRICS")
print("="*70)
print("\nTraining Metrics:")
for metric, value in lr_train_metrics.items():
    print(f"  {metric:20s}: {value:.4f}")

print("\nTest Metrics:")
for metric, value in lr_test_metrics.items():
    print(f"  {metric:20s}: {value:.4f}")
print("="*70)

---
## 5. Model 2: Support Vector Classifier (SVC)

Train SVC with RBF kernel for multi-class classification.

In [ ]:
print("="*70)
print("MODEL 2: LINEAR SUPPORT VECTOR CLASSIFIER (LinearSVC)")
print("="*70)
print("\nTraining LinearSVC (much faster than RBF kernel)...")
print("  Note: LinearSVC uses linear kernel, optimized for large datasets\n")

start_time = time.time()

# OPTIMIZATION: Use LinearSVC instead of SVC(kernel='rbf')
# LinearSVC is 50-100x faster and works well for this problem
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# LinearSVC doesn't support predict_proba, so we use calibration
# dual='auto' chooses optimal solver (False when samples >> features)
base_svc = LinearSVC(dual='auto', verbose=1, max_iter=2000, random_state=RANDOM_SEED)
svc_model = CalibratedClassifierCV(base_svc, cv=3, verbose=1)

# Train model with progress bar
with tqdm(total=100, desc='Training LinearSVC', bar_format='{l_bar}{bar}| {elapsed}') as pbar:
    svc_model.fit(X_train, y_train)
    pbar.update(100)

train_time = time.time() - start_time

# Store training time for comparison
svc_train_time = train_time
print(f"✓ Training completed in {train_time:.2f} seconds")

# Predictions
print("\nGenerating predictions...")
y_train_pred_svc = svc_model.predict(X_train)
y_test_pred_svc = svc_model.predict(X_test)

# Predict probabilities
y_train_proba_svc = svc_model.predict_proba(X_train)
y_test_proba_svc = svc_model.predict_proba(X_test)

print(f"\nPrediction shapes:")
print(f"  Train predictions: {y_train_pred_svc.shape}")
print(f"  Test predictions:  {y_test_pred_svc.shape}")


In [ ]:
# Compute metrics
svc_train_metrics = {
    'Accuracy': accuracy_score(y_train, y_train_pred_svc),
    'Precision (macro)': precision_score(y_train, y_train_pred_svc, average='macro', zero_division=0),
    'Recall (macro)': recall_score(y_train, y_train_pred_svc, average='macro', zero_division=0),
    'F1-Score (macro)': f1_score(y_train, y_train_pred_svc, average='macro', zero_division=0)
}

svc_test_metrics = {
    'Accuracy': accuracy_score(y_test, y_test_pred_svc),
    'Precision (macro)': precision_score(y_test, y_test_pred_svc, average='macro', zero_division=0),
    'Recall (macro)': recall_score(y_test, y_test_pred_svc, average='macro', zero_division=0),
    'F1-Score (macro)': f1_score(y_test, y_test_pred_svc, average='macro', zero_division=0)
}

print("\n" + "="*70)
print("SVC - METRICS")
print("="*70)
print("\nTraining Metrics:")
for metric, value in svc_train_metrics.items():
    print(f"  {metric:20s}: {value:.4f}")

print("\nTest Metrics:")
for metric, value in svc_test_metrics.items():
    print(f"  {metric:20s}: {value:.4f}")
print("="*70)

---
## 6. Model 3: PCA + Logistic Regression

Apply PCA for dimensionality reduction, then train Logistic Regression.

** Professor Requirement #3: Implement PCA**

In [ ]:
print("="*70)
print("MODEL 3: PCA + LOGISTIC REGRESSION")
print("="*70)
print("\nCreating PCA + Logistic Regression pipeline...\n")

start_time = time.time()

# Create pipeline
pca_lr_pipeline = Pipeline([
    ('pca', PCA(n_components=0.90, random_state=42)),  # Keep 90% variance (faster)
    ('lr', LogisticRegression(verbose=1, multi_class='multinomial', solver='saga', max_iter=500, random_state=42, n_jobs=-1))
])

# Train pipeline with progress bar
with tqdm(total=100, desc='Training PCA+LogReg', bar_format='{l_bar}{bar}| {elapsed}') as pbar:
    pca_lr_pipeline.fit(X_train, y_train)
    pbar.update(100)

train_time = time.time() - start_time

# Store for comparison
pca_lr_train_time = train_time
print(f"✓ Training completed in {train_time:.2f} seconds")

# Get number of components
n_components = pca_lr_pipeline.named_steps['pca'].n_components_
explained_variance = pca_lr_pipeline.named_steps['pca'].explained_variance_ratio_.sum()

print(f"\nPCA Details:")
print(f"  Original features:     {X_train.shape[1]}")
print(f"  PCA components:        {n_components}")
print(f"  Variance explained:    {explained_variance:.4f} (90% target)")
print(f"  Dimensionality reduction: {(1 - n_components/X_train.shape[1])*100:.1f}%")

# Predictions
print("\nGenerating predictions...")
y_train_pred_pca_lr = pca_lr_pipeline.predict(X_train)
y_test_pred_pca_lr = pca_lr_pipeline.predict(X_test)

# Predict probabilities
y_train_proba_pca_lr = pca_lr_pipeline.predict_proba(X_train)
y_test_proba_pca_lr = pca_lr_pipeline.predict_proba(X_test)


In [ ]:
# Compute metrics
pca_lr_train_metrics = {
    'Accuracy': accuracy_score(y_train, y_train_pred_pca_lr),
    'Precision (macro)': precision_score(y_train, y_train_pred_pca_lr, average='macro', zero_division=0),
    'Recall (macro)': recall_score(y_train, y_train_pred_pca_lr, average='macro', zero_division=0),
    'F1-Score (macro)': f1_score(y_train, y_train_pred_pca_lr, average='macro', zero_division=0)
}

pca_lr_test_metrics = {
    'Accuracy': accuracy_score(y_test, y_test_pred_pca_lr),
    'Precision (macro)': precision_score(y_test, y_test_pred_pca_lr, average='macro', zero_division=0),
    'Recall (macro)': recall_score(y_test, y_test_pred_pca_lr, average='macro', zero_division=0),
    'F1-Score (macro)': f1_score(y_test, y_test_pred_pca_lr, average='macro', zero_division=0)
}

print("\n" + "="*70)
print("PCA + LOGISTIC REGRESSION - METRICS")
print("="*70)
print("\nTraining Metrics:")
for metric, value in pca_lr_train_metrics.items():
    print(f"  {metric:20s}: {value:.4f}")

print("\nTest Metrics:")
for metric, value in pca_lr_test_metrics.items():
    print(f"  {metric:20s}: {value:.4f}")
print("="*70)

---
## 7. Model Comparison Table

Compare all models side-by-side.

In [ ]:
# Create comparison DataFrame
comparison_data = []

# Logistic Regression
comparison_data.append({
    'Model': 'Logistic Regression',
    'Training Time (s)': lr_train_time,
    'Train Accuracy': lr_train_metrics['Accuracy'],
    'Test Accuracy': lr_test_metrics['Accuracy'],
    'Train F1 (macro)': lr_train_metrics['F1-Score (macro)'],
    'Test F1 (macro)': lr_test_metrics['F1-Score (macro)'],
    'Test F1 (weighted)': f1_score(y_test, y_test_pred_lr, average='weighted'),
    'Test Precision (weighted)': precision_score(y_test, y_test_pred_lr, average='weighted'),
    'Test Recall (weighted)': recall_score(y_test, y_test_pred_lr, average='weighted')
})

# LinearSVC
comparison_data.append({
    'Model': 'LinearSVC',
    'Training Time (s)': svc_train_time,
    'Train Accuracy': svc_train_metrics['Accuracy'],
    'Test Accuracy': svc_test_metrics['Accuracy'],
    'Train F1 (macro)': svc_train_metrics['F1-Score (macro)'],
    'Test F1 (macro)': svc_test_metrics['F1-Score (macro)'],
    'Test F1 (weighted)': f1_score(y_test, y_test_pred_svc, average='weighted'),
    'Test Precision (weighted)': precision_score(y_test, y_test_pred_svc, average='weighted'),
    'Test Recall (weighted)': recall_score(y_test, y_test_pred_svc, average='weighted')
})

# PCA + Logistic Regression
comparison_data.append({
    'Model': 'PCA + LogReg',
    'Training Time (s)': pca_lr_train_time,
    'Train Accuracy': pca_lr_train_metrics['Accuracy'],
    'Test Accuracy': pca_lr_test_metrics['Accuracy'],
    'Train F1 (macro)': pca_lr_train_metrics['F1-Score (macro)'],
    'Test F1 (macro)': pca_lr_test_metrics['F1-Score (macro)'],
    'Test F1 (weighted)': f1_score(y_test, y_test_pred_pca_lr, average='weighted'),
    'Test Precision (weighted)': precision_score(y_test, y_test_pred_pca_lr, average='weighted'),
    'Test Recall (weighted)': recall_score(y_test, y_test_pred_pca_lr, average='weighted')
})

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*110)
print("MODEL PERFORMANCE COMPARISON (with Training Time & Weighted Metrics)")
print("="*110)
print(comparison_df.to_string(index=False))
print("="*110)

# Save comparison table
comparison_df.to_csv('../outputs/model_comparison.csv', index=False)
print("\n✓ Comparison table saved: outputs/model_comparison.csv")


---
## 8. Parity Plots

** Professor Requirement #5: Show parity plots for train and test data**

For multi-class classification, plot predicted probability vs. actual class membership for each class.

In [ ]:
def plot_parity_multiclass(y_true, y_pred_proba, model_name, class_labels, data_type="Test", max_classes=6):
    """
    Plot parity plots for multi-class classification.
    For each class, plot predicted probability vs. actual class membership.
    
    Parameters:
    -----------
    y_true : array
        True labels
    y_pred_proba : array
        Predicted probabilities (n_samples, n_classes)
    model_name : str
        Name of the model
    class_labels : list
        List of class labels
    data_type : str
        "Train" or "Test"
    max_classes : int
        Maximum number of classes to plot (to avoid overcrowding)
    """
    n_classes = min(len(class_labels), max_classes)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for i in range(n_classes):
        # Binary indicator: 1 if true class equals this class, else 0
        y_binary = (y_true == i).astype(int)
        
        # Predicted probability for this class
        y_proba_class = y_pred_proba[:, i]
        
        # Scatter plot with transparency
        axes[i].scatter(y_binary, y_proba_class, alpha=0.2, s=5, color='blue')
        axes[i].plot([0, 1], [0, 1], 'r--', linewidth=2, label='Perfect Prediction')
        axes[i].set_xlabel(f'Actual (Class {i})', fontsize=10)
        axes[i].set_ylabel('Predicted Probability', fontsize=10)
        axes[i].set_title(f'Class {i}: {list(class_labels.keys())[i][:15]}', fontsize=10, fontweight='bold')
        axes[i].set_xlim([-0.1, 1.1])
        axes[i].set_ylim([-0.1, 1.1])
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)
    
    plt.suptitle(f'{model_name} - {data_type} Parity Plots', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save plot
    filename = f"../outputs/parity_{model_name.lower().replace(' ', '_')}_{data_type.lower()}.png"
    plt.savefig(filename, dpi=DPI, bbox_inches='tight')
    plt.close()  # Free memory
    print(f" Saved: {filename}")

print(" Parity plot function defined")

In [ ]:
# Parity plots for Logistic Regression
print("\nGenerating parity plots for Logistic Regression...\n")

# Test data
plot_parity_multiclass(y_test, y_test_proba_lr, "Logistic Regression", label_mapping, data_type="Test")

# Train data
plot_parity_multiclass(y_train, y_train_proba_lr, "Logistic Regression", label_mapping, data_type="Train")

In [ ]:
# Parity plots for SVC
print("\nGenerating parity plots for SVC...\n")

# Test data
plot_parity_multiclass(y_test, y_test_proba_svc, "SVC", label_mapping, data_type="Test")

# Train data
plot_parity_multiclass(y_train, y_train_proba_svc, "SVC", label_mapping, data_type="Train")

In [ ]:
# Parity plots for PCA + Logistic Regression
print("\nGenerating parity plots for PCA + Logistic Regression...\n")

# Test data
plot_parity_multiclass(y_test, y_test_proba_pca_lr, "PCA + LogReg", label_mapping, data_type="Test")

# Train data
plot_parity_multiclass(y_train, y_train_proba_pca_lr, "PCA + LogReg", label_mapping, data_type="Train")

---
## 9. Confusion Matrices

** Professor Requirement #5 (Continued): Visualization of classification performance**

In [ ]:
# Plot confusion matrices for all models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_predictions = [
    ('Logistic Regression', y_test_pred_lr),
    ('SVC (RBF)', y_test_pred_svc),
    ('PCA + LogReg', y_test_pred_pca_lr)
]

for idx, (model_name, y_pred) in enumerate(models_predictions):
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                cbar_kws={'shrink': 0.8})
    axes[idx].set_title(f'{model_name}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Predicted Label', fontsize=10)
    axes[idx].set_ylabel('True Label', fontsize=10)

plt.suptitle('Confusion Matrices - Test Data', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(get_output_path('confusion_matrices.png', dpi=DPI, bbox_inches='tight')
plt.close()  # Free memory

print(" Saved: outputs/confusion_matrices.png")

---
## 10. Classification Reports

Detailed per-class performance metrics.

In [ ]:
# Get class names
class_names = list(label_mapping.keys())

print("="*90)
print("CLASSIFICATION REPORT - LOGISTIC REGRESSION")
print("="*90)
print(classification_report(y_test, y_test_pred_lr, target_names=class_names, zero_division=0))

print("\n" + "="*90)
print("CLASSIFICATION REPORT - SVC")
print("="*90)
print(classification_report(y_test, y_test_pred_svc, target_names=class_names, zero_division=0))

print("\n" + "="*90)
print("CLASSIFICATION REPORT - PCA + LOGISTIC REGRESSION")
print("="*90)
print(classification_report(y_test, y_test_pred_pca_lr, target_names=class_names, zero_division=0))

In [ ]:
# Save trained models for reproducibility
import joblib
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

print("Saving trained models...\n")

joblib.dump(lr_model, get_model_path('logistic_regression.pkl')
print("✓ Saved: models/logistic_regression.pkl")

joblib.dump(svc_model, get_model_path('linear_svc.pkl')
print("✓ Saved: models/linear_svc.pkl")

joblib.dump(pca_lr_pipeline, get_model_path('pca_lr_pipeline.pkl')
print("✓ Saved: models/pca_lr_pipeline.pkl")

print("\n✓ All models saved successfully!")


---
## 11. Summary & Key Findings

### Model Training Summary:
1.  Trained 3 models:
   - Logistic Regression (Multi-class, multinomial)
   - Support Vector Classifier (RBF kernel)
   - PCA + Logistic Regression (95% variance retained)

2.  Computed metrics:
   - Accuracy, Precision (macro), Recall (macro), F1-Score (macro)
   - Separate metrics for train and test sets

3.  Generated visualizations:
   - Parity plots for all 3 models (train and test)
   - Confusion matrices for all 3 models
   - Classification reports with per-class metrics

### Best Performing Model:
**Based on Test Accuracy:** [Fill after running - check comparison table]

### Observations:
- **Logistic Regression:** [Fill observations about speed, accuracy]
- **SVC:** [Fill observations about accuracy vs. training time]
- **PCA + LogReg:** [Fill observations about dimensionality reduction impact]

### Files Generated:
- `outputs/model_comparison.csv` - Metrics comparison table
- `outputs/parity_*.png` - Parity plots (6 files)
- `outputs/confusion_matrices.png` - Confusion matrices

---

**Next Steps for Member 4 (Cross-Validation):**
1. Perform 5-fold stratified cross-validation
2. Compare CV performance across all models
3. Analyze feature importance (Logistic Regression coefficients)
4. Simple hyperparameter tuning with GridSearchCV
5. Write final conclusions

---

**Proceed to:** `04_cross_validation_analysis.ipynb`